In [1]:
import os
os.chdir('..')

In [6]:
from databases.datasets import PersonStackedMucoTempDataset, PersonStackedMuPoTsDataset
from training.loaders import UnchunkedGenerator, ChunkedGenerator
from training.preprocess import get_postprocessor, SaveableCompose, MeanNormalize3D
from training.preprocess import decode_trfrm, RemoveIndex

In [3]:
LOG_PATH = "../models"
def extract_post(model_name, test_set):
    params_path = os.path.join(LOG_PATH, str(model_name), "preprocess_params.pkl")
    transform = SaveableCompose.from_file(params_path, test_set, globals())
    test_set.transform = transform

    assert isinstance(transform.transforms[1].normalizer, MeanNormalize3D)
    normalizer3d = transform.transforms[1].normalizer

    return get_postprocessor(config, test_set, normalizer3d)

In [4]:
config = {'adam_amsgrad': True,
 'batch_size': 1024,
 'cap_25fps': True,
 'learning_rate': 0.001,
 'lr_scheduler': {'multiplier': 0.95,
  'step_size': 1,
  'type': 'multiplicative'},
 'megadepth_type': 'megadepth_at_hrnet',
 'model': {'channels': 512,
  'dropout': 0.25,
  'filter_widths': [3, 3, 3],
  'layernorm': 'none',
  'loss': 'l1'},
 'num_epochs': 15,
 'optimiser': 'adam',
 'ordered_batch': False,
 'pose2d_type': 'hrnet',
 'pose3d_scaling': 'normal',
 'preprocess_2d': 'DepthposeNormalize2D',
 'preprocess_3d': 'SplitToRelativeAbsAndMeanNormalize3D',
 'sgd_momentum': 0,
 'shuffle': True,
 'simple_aug': True,
 'stride': 2,
 'test_time_flip': True,
 'train_data': 'mpii+muco',
 'train_time_flip': True}

train_data = PersonStackedMucoTempDataset(
    config["pose2d_type"], config["pose3d_scaling"]
)
eval_data = PersonStackedMuPoTsDataset(
    config["pose2d_type"],
    config.get("pose3d_scaling", "normal"),
    pose_validity="all",
)

In [7]:
train_data.transform = None
transforms_train = [
    decode_trfrm(config["preprocess_2d"], globals())(train_data, cache=False),
    decode_trfrm(config["preprocess_3d"], globals())(train_data, cache=False),
]

transforms_train.append(RemoveIndex())
train_data.transform = SaveableCompose(transforms_train)


In [8]:
extract_post("29cbfa0fc1774b9cbb06a3573b7fb711", eval_data)

<function training.preprocess.get_postprocessor.<locals>.f(x, seq)>

In [9]:
pad = 27
train_gen = ChunkedGenerator(
    train_data,
    config["batch_size"],
    pad,
    config["train_time_flip"],
    shuffle=config["shuffle"],
    ordered_batch=config["ordered_batch"],
)
eval_gen = UnchunkedGenerator(eval_data, pad, True)

In [10]:
t = next(iter(train_gen))

In [11]:
e = next(iter(eval_gen))

In [12]:
e[0].shape, e[1].shape

((2, 255, 42), (2, 201))

In [14]:
t.keys()

dict_keys(['temporal_pose2d', 'pose3d', 'valid_pose'])

In [17]:
t['temporal_pose2d'].shape, t['pose3d'].shape, t['valid_pose'].shape

(torch.Size([1024, 55, 42]), torch.Size([1024, 1, 51]), torch.Size([1024]))